In [18]:
import logging
import os
import time
from datetime import datetime
import maritalk
import pandas as pd
from dotenv import load_dotenv

In [19]:
load_dotenv()

name = 'oneshot-teste1'
pathDataset = '../datasets/dataset-maritaca-oneshot-teste1.csv'
chave = os.getenv('MARITAKA_KEY')
model_engine = "sabia-2-medium"

In [20]:
logging.basicConfig(filename=f'./logs/maritaca-{name}.log', level=logging.INFO, encoding='utf8')
logging.info(f'LOG DO DIA {datetime.now()}')
logging.info(f'TESTANDO O DATASET {pathDataset}')

In [21]:
df_maritaca = pd.read_csv(pathDataset)

size = len(df_maritaca)
print ('Quantidade de comentários:', size)

Quantidade de comentários: 4000


In [29]:
for i in range(3500, 4000):
    sentence = df_maritaca['sentenca'][i]
    prompt= f'''Você é um especialista em restaurantes. A partir de um comentário contendo a opinião de um cliente sobre um restaurante, identifique os seguintes temas contidos nesse comentário:
                serviço: Representa sentenças relacionadas às atitudes dos atendentes, presteza, interação com o gerente ou atraso. Exemplo: "O serviço no restaurante foi excepcional, com atendentes que demonstravam um entusiasmo genuíno e a gerência presente, sempre atenta às necessidades dos clientes, proporcionando uma experiência agradável e sem atrasos."
                bebida: Representa sentenças relacionadas a cerveja, vinho, drinks, cafés entre outras bebidas do restaurante. Exemplo: "A cerveja estava extremamente gelada e o vinho possuía uma excelente qualidade, o que tornou a experiência no restaurante ainda mais agradável."
                ambiente: Representa sentenças relacionadas a decoração, interior, música, espaço, iluminação ou lotação. Exemplo: "O ambiente do restaurante era acolhedor, com uma decoração rústica e aconchegante, música suave ao fundo, espaço bem distribuído e iluminação adequada, proporcionando um clima ideal para uma refeição tranquila."  
                comida: Representa sentenças relacionadas à alimentação,  sobremesa, cardápio, rodízio, sabor da comida ou pratos. Exemplo: "A comida estava deliciosa, com sabores autênticos e pratos bem apresentados, tornando a experiência gastronômica memorável."
                localização: Representa sentenças relacionadas ao transporte ou o acesso ao restaurante, estacionamento, entre outros. Exemplo: "O restaurante está situado em um local privilegiado, com fácil acesso por diversas linhas de transporte público e conta com um estacionamento próprio amplo, o que é um grande atrativo para quem prefere ir de carro."
                preço: Representa sentenças relacionadas ao custo do serviço ou dos produtos dos restaurantes. Exemplo: "Os preços dos pratos são justos, considerando a qualidade e a quantidade do que é oferecido."
                geral: Representa sentenças relacionadas à uma visão abrangente do restaurante como um todo, sem se concentrar em características específicas, como comida, serviço ou ambiente. Esses comentários expressam uma opinião geral sobre o restaurante, sem levar em consideração aspectos particulares. Exemplo: "Eu gostaria de expressar minha satisfação geral com a experiência no restaurante; desde a chegada até a saída, tudo foi muito agradável e bem organizado."
                outros: Representa sentenças relacionadas que não se encaixam em nenhum tema citado anteriormente.

                Note que uma sentença pode conter mais de um tema identificado.

                Retorne uma saída contendo APENAS os nomes dos temas identificados na sentença.
                Exemplo 
                Entrada: <FRASE>
                comida, bebida, serviço

                Analise a frase: '{sentence}' '''
    
    try:
        model = maritalk.MariTalk(
            key=chave,
            model=model_engine  # No momento, suportamos os modelos sabia-2-medium e sabia-2-small
        )
    except:
        time.sleep(15)
        model = maritalk.MariTalk(
            key=chave,
            model=model_engine  # No momento, suportamos os modelos sabia-2-medium e sabia-2-small
        )
        
    fullResponse = model.generate(prompt)
    response = fullResponse['answer'].replace('.', '').lower().strip()

    # Atualizar o dataset novo com os temas identificados
    df_maritaca.loc[i, 'categoria_maritaka'] = response

    logging.info('SENTENÇA: ' + sentence)
    logging.info('CHAT: ', fullResponse)
    logging.info('RESPONSE: ' + response)
    
    # Salvar o dataframe atualizado com as categorias identificadas
    df_maritaca.to_csv(f'../datasets/dataset-maritaca-{name}.csv', index=False)
    
    time.sleep(3)
    


In [23]:
sentenca = 'E ainda tem uma brinquedoteca espetacular, por isso as crianças adoram ir lá.'
linha = df_maritaca.loc[df_maritaca['sentenca'] == sentenca]
linha

,sentenca,categoria_maritaka
3571,"E ainda tem uma brinquedoteca espetacular, por...",NaN
